In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import re

from nltk.corpus import stopwords
stop = list(set(stopwords.words('english')))

In [2]:
f_train = open('../data/train_14k_split_conll.txt','r',encoding='utf8')
line_train = f_train.readlines()

f_val = open('../data/dev_3k_split_conll.txt','r',encoding='utf8')
line_val = f_val.readlines()

f_test = open('../data/Hindi_test_unalbelled_conll_updated.txt','r',encoding='utf8')
line_test = f_test.readlines()

In [3]:
def get_data(lines):
    doc_count = 0
    i = 0
    #global train_word, train_word_type, train_doc_sentiment, train_uid, train_doc_id
    train_word = []
    train_word_type = []
    train_doc_sentiment = []
    train_uid = []
    train_doc_id = []

    while i < len(lines):
        '''
        if i < len(lines) and lines[i].split('\t')[0] in ['http','https']:
            i += 1
            while i < len(lines) and lines[i].split('\t')[0] != '/':
                i += 1
                
            i += 1    
        ''' 

        if i < len(lines) and lines[i].split('\t')[0] in ['@']:
            i += 2

        if i < len(lines):
            line = lines[i]
            line = line.replace('\n','').strip().lower()
            if line.split('\t')[0] == 'meta':
                doc_count += 1
                uid = int(line.split('\t')[1])
                sentiment = line.split('\t')[2]
                #if doc_count == 1575:
                #    print (uid)
                i += 1

            elif len(line.split('\t')) >= 2:
                if line.split('\t')[0] not in stop:
                    train_uid.append(uid)
                    train_doc_sentiment.append(sentiment)
                    train_word.append(line.split('\t')[0])
                    train_word_type.append(line.split('\t')[1])
                    train_doc_id.append(doc_count)

                i += 1

            else:
                i += 1

    train_df = pd.DataFrame()
    train_df['doc_id'] = train_doc_id
    train_df['word'] = train_word
    train_df['word_type'] = train_word_type
    train_df['uid'] = train_uid
    train_df['sentiment'] = train_doc_sentiment
    
    return train_df

In [4]:
def get_data_test(lines):
    doc_count = 0
    i = 0
    #global train_word, train_word_type, train_doc_sentiment, train_uid, train_doc_id
    train_word = []
    train_word_type = []
    train_uid = []
    train_doc_id = []

    while i < len(lines):
        '''
        if i < len(lines) and lines[i].split('\t')[0] in ['http','https']:
            i += 1
            while i < len(lines) and lines[i].split('\t')[0] != '/':
                i += 1
                
            i += 1    
        ''' 

        if i < len(lines) and lines[i].split('\t')[0] in ['@']:
            i += 2

        if i < len(lines):
            line = lines[i]
            line = line.replace('\n','').strip().lower()
            if line.split('\t')[0] == 'meta':
                doc_count += 1
                uid = int(line.split('\t')[1])
                #if doc_count == 1575:
                #    print (uid)
                i += 1

            elif len(line.split('\t')) >= 2:
                if line.split('\t')[0] not in stop:
                    train_uid.append(uid)
                    train_word.append(line.split('\t')[0])
                    train_word_type.append(line.split('\t')[1])
                    train_doc_id.append(doc_count)

                i += 1

            else:
                i += 1

    train_df = pd.DataFrame()
    train_df['doc_id'] = train_doc_id
    train_df['word'] = train_word
    train_df['word_type'] = train_word_type
    train_df['uid'] = train_uid
    
    return train_df

In [5]:
train_df = get_data(line_train)
val_df = get_data(line_val)
test_df = get_data_test(line_test)

In [6]:
train_df.shape, val_df.shape, test_df.shape

((289253, 5), (62618, 5), (61735, 4))

In [7]:
train_df.uid.nunique(), val_df.uid.nunique(), test_df.uid.nunique()

(14000, 3000, 3000)

In [8]:
train_df = train_df[train_df.word != 'http']
train_df = train_df[train_df.word != 'https']
#train_df = train_df[train_df.word_type != 'o']
print (train_df.shape)

val_df = val_df[val_df.word != 'http']
val_df = val_df[val_df.word != 'https']
#val_df = val_df[val_df.word_type != 'o']
print (val_df.shape)

test_df = test_df[test_df.word != 'http']
test_df = test_df[test_df.word != 'https']
#test_df = test_df[test_df.word_type != 'o']
print (test_df.shape)

(281408, 5)
(60908, 5)
(60121, 4)


In [9]:
"""
train_df.word = train_df.word.apply(lambda x: re.sub("[^a-zA-Z0-9]", "",x))
train_df = train_df[train_df.word.str.len() >= 3]
print (train_df.shape)

val_df.word = val_df.word.apply(lambda x: re.sub("[^a-zA-Z0-9]", "",x))
val_df = val_df[val_df.word.str.len() >= 3]
print (val_df.shape)

test_df.word = test_df.word.apply(lambda x: re.sub("[^a-zA-Z0-9]", "",x))
test_df = test_df[test_df.word.str.len() >= 3]
print (test_df.shape)
"""

'\ntrain_df.word = train_df.word.apply(lambda x: re.sub("[^a-zA-Z0-9]", "",x))\ntrain_df = train_df[train_df.word.str.len() >= 3]\nprint (train_df.shape)\n\nval_df.word = val_df.word.apply(lambda x: re.sub("[^a-zA-Z0-9]", "",x))\nval_df = val_df[val_df.word.str.len() >= 3]\nprint (val_df.shape)\n\ntest_df.word = test_df.word.apply(lambda x: re.sub("[^a-zA-Z0-9]", "",x))\ntest_df = test_df[test_df.word.str.len() >= 3]\nprint (test_df.shape)\n'

In [10]:
all_words = set(pd.concat([train_df[['word']],val_df[['word']],test_df[['word']]], axis=0).word)
print ("Total number of words {}".format(len(all_words)))

Total number of words 55793


In [11]:
all_words = pd.concat([train_df[['word']],val_df[['word']],test_df[['word']]], axis=0)
all_words = all_words.word.value_counts().reset_index()
all_words.columns = ['word','tot_count']
top_words = all_words[all_words.tot_count >= 3]
print (top_words.shape)
print (top_words.head(5))

(11414, 2)
  word  tot_count
0    .      13905
1    /      11263
2   //      11138
3   co      10872
4    …       9194


In [12]:
#pd.DataFrame(['[PAD]','[UNK]','[CLS]','[SEP]'],columns=['word'])
f = open('../bert-base-uncased-vocab.txt')
ll = [i.replace('\n','') for i in f.readlines()]
all_bert_words = pd.DataFrame(ll,columns=['word'])

all_bert_words = pd.concat([all_bert_words,top_words[['word']].drop_duplicates().reset_index(drop=True)],axis=0)
all_bert_words = all_bert_words.drop_duplicates().reset_index(drop=True)
all_bert_words.to_csv("../bert_vocab.txt",index=False,header=False)

In [13]:
train_texts = train_df.groupby(['uid'],sort=True)['word'].apply(lambda x: " ".join(x)).reset_index()
train_texts = pd.merge(train_texts,train_df[['uid','sentiment']],how='left').drop_duplicates().reset_index(drop=True)
train_texts.columns = ['uid','text','sentiment']
#train_texts.text = train_texts.text.apply(lambda x: '[CLS] ' + x + ' [SEP]')

val_texts = val_df.groupby(['uid'],sort=True)['word'].apply(lambda x: " ".join(x)).reset_index().reset_index(drop=True)
val_texts = pd.merge(val_texts,val_df[['uid','sentiment']],how='left').drop_duplicates()
val_texts.columns = ['uid','text','sentiment']
#val_texts.text = val_texts.text.apply(lambda x: '[CLS] ' + x + ' [SEP]')

test_texts = test_df.groupby(['uid'],sort=True)['word'].apply(lambda x: " ".join(x)).reset_index().reset_index(drop=True)
test_texts.columns = ['uid','text']
#test_texts.text = test_texts.text.apply(lambda x: '[CLS] ' + x + ' [SEP]')


In [14]:
train_texts.head(5)

,uid,text,sentiment
0,8,rt love looks good maddie !!! ako lang ba yung...,neutral
1,14,@ min _ _ lyching @ manakgupta mein kahna nae ...,neutral
2,26,best luck sir world cup ke liye bhot bhot subh...,positive
3,27,yes . great dialogues one . also chupke chupke...,positive
4,33,@ tarekfatah tu tere baap ke liye jo bola wo k...,negative


In [15]:
test_texts.head(5)

,uid,text
0,1,keh aese rahe ho jaise pakistan wale ni karte ...
1,4,rt anu's prerna way ran saving ... ram ram jay...
2,17,shukar hai ye pathan nae warna # ptm nay pr b ...
3,18,harsh pen decision . much option arm . big sec...
4,34,rt keep saying kenyan rugby beautiful save kru...


In [16]:
all_texts = pd.concat([train_texts[['uid','text','sentiment']],val_texts[['uid','text','sentiment']]],axis=0)
all_texts.sentiment.value_counts()

neutral     6392
positive    5616
negative    4992
Name: sentiment, dtype: int64

In [17]:
from transformers import BertTokenizer, BertConfig, BertModel
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

I0227 11:19:33.086394 4540767680 file_utils.py:38] PyTorch version 1.2.0 available.
Using TensorFlow backend.


In [18]:
tokenizer = BertTokenizer('../bert_vocab.txt')

In [19]:
tokenizer.vocab['karnal']

36847

In [20]:
tokenizer.vocab_size

38126

In [21]:
train_df.groupby(['uid'])['word'].count().describe()

count    14000.000000
mean        20.100571
std          6.089602
min          1.000000
25%         16.000000
50%         21.000000
75%         25.000000
max         48.000000
Name: word, dtype: float64

In [22]:
max_len = 40

In [23]:
n_output = all_texts.sentiment.nunique() #number of possible outputs
senti_dict = {'negative':0,'neutral':1,'positive':2}
in_senti_dict = {0:'negative',1:'neutral',2:'positive'}
all_texts.sentiment = all_texts.sentiment.apply(lambda x: senti_dict[x])

In [24]:
train_tokens = []
for text in tqdm(all_texts.text.values.tolist()):
    train_tokens += [tokenizer.encode(text,add_special_tokens=False)]

100%|██████████| 17000/17000 [00:05<00:00, 2835.29it/s]


In [25]:
test_tokens = []
for text in tqdm(test_texts.text.values.tolist()):
    test_tokens += [tokenizer.encode(text,add_special_tokens=False)]

100%|██████████| 3000/3000 [00:01<00:00, 2648.23it/s]


In [26]:
train_tokens_padded = []
train_attention_mask = []
train_seg_ids = []

for tokens in tqdm(train_tokens):
    tokens = tokens[:max_len]
    token_len = len(tokens)
    one_mask = [1]*token_len
    zero_mask = [0]*(max_len-token_len)
    padded_input = tokens + zero_mask
    attention_mask = one_mask + zero_mask
    
    segments = []
    first_sep = True
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == 3:
            current_segment_id = 1
    segments = segments + [0] * (max_len - len(tokens))
    
    train_tokens_padded += [padded_input]
    train_attention_mask += [attention_mask]
    train_seg_ids += [segments]

100%|██████████| 17000/17000 [00:00<00:00, 67878.17it/s]


In [27]:
test_tokens_padded = []
test_attention_mask = []
test_seg_ids = []

for tokens in tqdm(test_tokens):
    tokens = tokens[:max_len]
    token_len = len(tokens)
    one_mask = [1]*token_len
    zero_mask = [0]*(max_len-token_len)
    padded_input = tokens + zero_mask
    attention_mask = one_mask + zero_mask
    
    segments = []
    first_sep = True
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == 102:
            current_segment_id = 1
    segments = segments + [0] * (max_len - len(tokens))
    
    test_tokens_padded += [padded_input]
    test_attention_mask += [attention_mask]
    test_seg_ids += [segments]

100%|██████████| 3000/3000 [00:00<00:00, 98383.17it/s]


In [28]:
print (train_tokens_padded[0], train_attention_mask[0], train_seg_ids[0])

[19387, 2293, 3504, 2204, 17805, 999, 999, 999, 31965, 11374, 8670, 32443, 37041, 37869, 32777, 2089, 37565, 4135, 36791, 1029, 2627, 1062, 19960, 7677, 2659, 14839, 1529, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [29]:
train_output = torch.LongTensor(to_categorical(all_texts.sentiment))

In [30]:
train_tokens_padded = torch.LongTensor(np.asarray(train_tokens_padded))
train_attention_mask = torch.LongTensor(np.asarray(train_attention_mask))
train_seg_ids = torch.LongTensor(np.asarray(train_seg_ids))
test_tokens_padded = torch.LongTensor(np.asarray(test_tokens_padded))
test_attention_mask = torch.LongTensor(np.asarray(test_attention_mask))
test_seg_ids = torch.LongTensor(np.asarray(test_seg_ids))

In [31]:
print (train_tokens_padded.shape, train_attention_mask.shape, train_seg_ids.shape, test_tokens_padded.shape, test_attention_mask.shape, test_seg_ids.shape)


torch.Size([17000, 40]) torch.Size([17000, 40]) torch.Size([17000, 40]) torch.Size([3000, 40]) torch.Size([3000, 40]) torch.Size([3000, 40])


In [32]:
dev_tokens_padded = train_tokens_padded[train_texts.shape[0]:]
train_tokens_padded = train_tokens_padded[:train_texts.shape[0]]

dev_attention_mask = train_attention_mask[train_texts.shape[0]:]
train_attention_mask = train_attention_mask[:train_texts.shape[0]]

dev_seg_ids = train_seg_ids[train_texts.shape[0]:]
train_seg_ids = train_seg_ids[:train_texts.shape[0]]

dev_output = train_output[train_texts.shape[0]:]
train_output = train_output[:train_texts.shape[0]]

In [33]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score, accuracy_score

In [34]:
batch_size = 32

In [35]:
train_data = TensorDataset(train_tokens_padded, train_attention_mask, train_seg_ids, train_output)
val_data = TensorDataset(dev_tokens_padded, dev_attention_mask, dev_seg_ids, dev_output)

#dataloader
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

In [36]:
class berttuned_model(nn.Module):
    def __init__(self, n_output=3, dropout_prob = .3, hidden_size=768):
        super().__init__()
        
        self.n_output = n_output
        
        config = BertConfig(vocab_size=tokenizer.vocab_size)
        
        self.basemodel = BertModel(config=config)
        
        self.fc = torch.nn.Linear(hidden_size,n_output)
        
        self.dropout = torch.nn.Dropout(dropout_prob)
        
    def forward(self, ids, attention_mask, seg_ids):
        bert_outputs = self.basemodel(input_ids=ids, token_type_ids=seg_ids, attention_mask=attention_mask)
        out = bert_outputs[0]
        out = torch.mean(out,1)
        out = self.fc(self.dropout(out))
        
        return out

In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = berttuned_model()
epochs = 20

In [38]:
model

berttuned_model(
  (basemodel): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(38126, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

In [39]:
print ("Total number of parameters to learn {}".format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

Total number of parameters to learn 115324419


In [40]:
optimizer = torch.optim.Adam(model.parameters(),lr=.00005)
criterion = torch.nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [41]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    if device == 'cpu':
        rounded_preds = preds.detach().numpy().argmax(1)
        rounded_correct = y.detach().numpy().argmax(1)
    else:
        rounded_preds = preds.detach().cpu().numpy().argmax(1)
        rounded_correct = y.detach().cpu().numpy().argmax(1)
    
    return accuracy_score(rounded_correct,rounded_preds)

In [42]:
def f1_torch(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    if device == 'cpu':
        rounded_preds = preds.detach().numpy().argmax(1)
        rounded_correct = y.detach().numpy().argmax(1)
    else:
        rounded_preds = preds.detach().cpu().numpy().argmax(1)
        rounded_correct = y.detach().cpu().numpy().argmax(1)
    
    return f1_score(rounded_correct,rounded_preds,average='macro')

In [43]:
def train(model, train_loader, optimizer, criterion):
    global predictions, labels, loss
    
    epoch_loss = 0
    epoch_acc = 0
    f1_scores = 0
    
    model.train()
    
    counter = 0
    for tokens, attn, seg, labels in tqdm(train_loader):
        
        counter += 1
        
        optimizer.zero_grad()
        
        predictions = model(tokens, attn, seg) #.squeeze(1)
        predictions = torch.softmax(predictions,dim=-1)
        
        #loss = criterion(predictions, labels)
        loss = criterion(predictions, torch.max(labels, 1)[1])
        
        acc = binary_accuracy(predictions, labels)
        f1_score_batch = f1_torch(predictions, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc
        f1_scores += f1_score_batch
        
    return epoch_loss / counter, epoch_acc / counter, f1_scores/counter

In [44]:
def evaluate(model, val_loader, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    f1_scores = 0
    
    model.eval()
    
    counter = 0
    with torch.no_grad():
        for tokens, attn, seg, labels in tqdm(val_loader):

            counter += 1


            predictions = model(tokens, attn, seg) #.squeeze(1)
            predictions = torch.softmax(predictions,dim=-1)
            
            #loss = criterion(predictions, labels)
            loss = criterion(predictions, torch.max(labels, 1)[1])

            acc = binary_accuracy(predictions, labels)
            f1_score_batch = f1_torch(predictions, labels)
            
            epoch_loss += loss.item()
            epoch_acc += acc
            f1_scores += f1_score_batch
        
    return epoch_loss / counter, epoch_acc / counter, f1_scores/counter

In [45]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [46]:
best_valid_loss = 999

for epoch in range(epochs):
    
    start_time = time.time()
    
    train_loss, train_acc, train_f1 = train(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc, valid_f1 = evaluate(model, val_loader, criterion)
       
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '../models/model_bert.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Train F1: {train_f1*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | Val F1: {valid_f1*100:.2f}%')

  0%|          | 0/110 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 64m 52s
	Train Loss: 1.135 | Train Acc: 35.52% | Train F1: 21.48%
	 Val. Loss: 1.114 |  Val. Acc: 38.59% | Val F1: 26.78%


  0%|          | 0/110 [00:00<?, ?it/s]

Epoch: 02 | Epoch Time: 144m 24s
	Train Loss: 1.100 | Train Acc: 36.65% | Train F1: 28.58%
	 Val. Loss: 1.082 |  Val. Acc: 38.09% | Val F1: 20.81%


  0%|          | 0/110 [00:00<?, ?it/s]

...

Epoch: 06 | Epoch Time: 84m 57s
	Train Loss: 0.867 | Train Acc: 67.62% | Train F1: 67.60%
	 Val. Loss: 1.063 |  Val. Acc: 47.67% | Val F1: 45.54%


  0%|          | 0/110 [00:00<?, ?it/s]

Epoch: 07 | Epoch Time: 84m 36s
	Train Loss: 0.923 | Train Acc: 62.01% | Train F1: 61.13%
	 Val. Loss: 1.151 |  Val. Acc: 38.43% | Val F1: 30.84%


 16%|█▋        | 18/110 [13:27<1:10:42, 46.12s/it]I0227 22:21:39.000735 4540767680 utils.py:141] NumExpr defaulting to 4 threads.


KeyboardInterrupt: 